In [1]:
%matplotlib
%matplotlib inline
import numpy
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
print(sys.path)

['', 'C:\\Users\\seanm\\Anaconda3\\python36.zip', 'C:\\Users\\seanm\\Anaconda3\\DLLs', 'C:\\Users\\seanm\\Anaconda3\\lib', 'C:\\Users\\seanm\\Anaconda3', 'C:\\Users\\seanm\\Anaconda3\\lib\\site-packages', 'C:\\Users\\seanm\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\seanm\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\seanm\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\seanm\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\seanm\\.ipython', 'C:\\Users\\seanm\\OneDrive\\Documents\\Research\\HybridSolver\\GillesPy2']


In [3]:
import math
import gillespy2
from gillespy2.basic_hybrid_solver import BasicHybridSolver

Unable to use Cython optimized SSA:
Error:Building module gillespy2.cython_ssa_solver failed: ['distutils.errors.DistutilsPlatformError: Unable to find vcvarsall.bat\n']


In [4]:

class SimpleHybridModel(gillespy2.Model):
     def __init__(self, parameter_values=None):
            #initialize Model
            gillespy2.Model.__init__(self, name="Simple_Hybrid_Model")

            
            #Species
            A = gillespy2.Species(name='A', initial_value=0)
            V = gillespy2.Species(name='V', initial_value=0, deterministic=True)

            self.add_species([A, V])
            
            #parameters
            rate1 = gillespy2.Parameter(name='rate1', expression= 20.0)
            rate2 = gillespy2.Parameter(name='rate2', expression= 10.0)
            rate_rule1 = gillespy2.RateRule(V, "sin(t)")
            self.add_parameter([rate1, rate2])
            self.add_rate_rule(rate_rule1)
            
            #reactions
            r1 = gillespy2.Reaction(name="r1",reactants={}, products={A:1},
                   propensity_function="rate1 * V")
            
            r2 = gillespy2.Reaction(name="r2",reactants={A:1}, products={},
                    rate=rate2)
            
            self.add_reaction([r1,r2])
            self.timespan(numpy.linspace(0,100,101))

In [5]:
model = SimpleHybridModel()

Adding rate rule  <gillespy2.gillespy2.RateRule object at 0x000001F76AA85EF0>  to  Simple_Hybrid_Model


In [6]:
%time results = model.run(solver=BasicHybridSolver(), show_labels=True, debug=True)

t =  100.0
increment =  1.0
Setting Random number  -0.08782387094018652  for  r1
Setting Random number  -1.2230465123498884  for  r2
Propensity of  r1  is 0
Propensity of  r2  is 0
NO projected reaction
Curr Time:  0  Save time:  1.0 step:  1.0
start time:  0 step taken:  1.0 number of steps: 11
Vode terminated at t=0.1
Current:  [-0.08782387 -1.22304651  0.00499578] 
curr_time:  0.1
AFTER: curr_time:  0.1  current:  [-0.08782387 -1.22304651  0.00499578]
Step exceeds save_time, changing step size from  1.0  to  0.9
Curr Time:  0.1  Save time:  1.0 step:  0.9
start time:  0.1 step taken:  1.0 number of steps: 11
Vode terminated at t=0.2
Current:  [-0.08782387 -1.22304651  0.01493758] 
curr_time:  0.2
AFTER: curr_time:  0.2  current:  [-0.08782387 -1.22304651  0.01493758]
No reactions fired in this step (n= 0 ) changing step size from  0.9  to  1.125
Step exceeds save_time, changing step size from  1.125  to  0.8
Curr Time:  0.2  Save time:  1.0 step:  0.8
start time:  0.2 step taken:  1

start time:  1.2114179592074656 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.21648
Current:  [-0.07904148 -1.22304651  0.09145161] 
curr_time:  1.216482411898714
AFTER: curr_time:  1.216482411898714  current:  [-0.07904148 -1.22304651  0.09145161]
No reactions fired in this step (n= 0 ) changing step size from  0.7885820407925344  to  0.985727550990668
Step exceeds save_time, changing step size from  0.985727550990668  to  0.7835175881012859
Curr Time:  1.216482411898714  Save time:  2.0 step:  0.7835175881012859
start time:  1.216482411898714 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.22155
Current:  [-0.07904148 -1.22304651  0.09146051] 
curr_time:  1.2215468645899625
AFTER: curr_time:  1.2215468645899625  current:  [-0.07904148 -1.22304651  0.09146051]
No reactions fired in this step (n= 0 ) changing step size from  0.7835175881012859  to  0.9793969851266073
Step exceeds save_time, changing step size from  0.97939698512

Step exceeds save_time, changing step size from  0.7958105750688513  to  0.6315840073638326
Curr Time:  1.3684159926361674  Save time:  2.0 step:  0.6315840073638326
start time:  1.3684159926361674 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.37348
Current:  [-0.07904148 -1.22304651  0.09166985] 
curr_time:  1.3734804453274159
AFTER: curr_time:  1.3734804453274159  current:  [-0.07904148 -1.22304651  0.09166985]
No reactions fired in this step (n= 0 ) changing step size from  0.6315840073638326  to  0.7894800092047907
Step exceeds save_time, changing step size from  0.7894800092047907  to  0.6265195546725841
Curr Time:  1.3734804453274159  Save time:  2.0 step:  0.6265195546725841
start time:  1.3734804453274159 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.37854
Current:  [-0.07904148 -1.22304651  0.09167488] 
curr_time:  1.3785448980186643
AFTER: curr_time:  1.3785448980186643  current:  [-0.07904148 -1.22304651  0.09167488

start time:  1.5304784787561176 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.53554
Current:  [-0.07904148 -1.22304651  0.09176704] 
curr_time:  1.535542931447366
AFTER: curr_time:  1.535542931447366  current:  [-0.07904148 -1.22304651  0.09176704]
No reactions fired in this step (n= 0 ) changing step size from  0.46952152124388236  to  0.5869019015548529
Step exceeds save_time, changing step size from  0.5869019015548529  to  0.4644570685526339
Curr Time:  1.535542931447366  Save time:  2.0 step:  0.4644570685526339
start time:  1.535542931447366 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.54061
Current:  [-0.07904148 -1.22304651  0.09176794] 
curr_time:  1.5406073841386145
AFTER: curr_time:  1.5406073841386145  current:  [-0.07904148 -1.22304651  0.09176794]
No reactions fired in this step (n= 0 ) changing step size from  0.4644570685526339  to  0.5805713356907924
Step exceeds save_time, changing step size from  0.58057133

Current:  [-0.07904148 -1.22304651  0.09173154] 
curr_time:  1.6976054175673163
AFTER: curr_time:  1.6976054175673163  current:  [-0.07904148 -1.22304651  0.09173154]
No reactions fired in this step (n= 0 ) changing step size from  0.30745903512393213  to  0.38432379390491517
Step exceeds save_time, changing step size from  0.38432379390491517  to  0.3023945824326837
Curr Time:  1.6976054175673163  Save time:  2.0 step:  0.3023945824326837
start time:  1.6976054175673163 step taken:  0.05064452691248475 number of steps: 11
Vode terminated at t=1.70267
Current:  [-0.07904148 -1.22304651  0.0917283 ] 
curr_time:  1.7026698702585648
AFTER: curr_time:  1.7026698702585648  current:  [-0.07904148 -1.22304651  0.0917283 ]
No reactions fired in this step (n= 0 ) changing step size from  0.3023945824326837  to  0.3779932280408546
Step exceeds save_time, changing step size from  0.3779932280408546  to  0.29733012974143525
Curr Time:  1.7026698702585648  Save time:  2.0 step:  0.29733012974143525

...

start time:  6.0890605540725815 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.09213
Current:  [-0.04667321 -1.22304651  0.08401622] 
curr_time:  6.092125411826431
AFTER: curr_time:  6.092125411826431  current:  [-0.04667321 -1.22304651  0.08401622]
No reactions fired in this step (n= 0 ) changing step size from  0.9109394459274185  to  1.1386743074092731
Step exceeds save_time, changing step size from  1.1386743074092731  to  0.907874588173569
Curr Time:  6.092125411826431  Save time:  7.0 step:  0.907874588173569
start time:  6.092125411826431 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.09519
Current:  [-0.04667321 -1.22304651  0.08402544] 
curr_time:  6.09519026958028
AFTER: curr_time:  6.09519026958028  current:  [-0.04667321 -1.22304651  0.08402544]
No reactions fired in this step (n= 0 ) changing step size from  0.907874588173569  to  1.1348432352169613
Step exceeds save_time, changing step size from  1.13484323521696

Current:  [-0.04667321 -1.22304651  0.08431359] 
curr_time:  6.190200859949614
AFTER: curr_time:  6.190200859949614  current:  [-0.04667321 -1.22304651  0.08431359]
No reactions fired in this step (n= 0 ) changing step size from  0.8128639978042358  to  1.0160799972552947
Step exceeds save_time, changing step size from  1.0160799972552947  to  0.8097991400503863
Curr Time:  6.190200859949614  Save time:  7.0 step:  0.8097991400503863
start time:  6.190200859949614 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.19327
Current:  [-0.04667321 -1.22304651  0.08432294] 
curr_time:  6.193265717703463
AFTER: curr_time:  6.193265717703463  current:  [-0.04667321 -1.22304651  0.08432294]
No reactions fired in this step (n= 0 ) changing step size from  0.8097991400503863  to  1.012248925062983
Step exceeds save_time, changing step size from  1.012248925062983  to  0.8067342822965369
Curr Time:  6.193265717703463  Save time:  7.0 step:  0.8067342822965369
start time: 

No reactions fired in this step (n= 0 ) changing step size from  0.7147885496810531  to  0.8934856871013164
Step exceeds save_time, changing step size from  0.8934856871013164  to  0.7117236919272036
Curr Time:  6.288276308072796  Save time:  7.0 step:  0.7117236919272036
start time:  6.288276308072796 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.29134
Current:  [-0.04667321 -1.22304651  0.08462314] 
curr_time:  6.291341165826646
AFTER: curr_time:  6.291341165826646  current:  [-0.04667321 -1.22304651  0.08462314]
No reactions fired in this step (n= 0 ) changing step size from  0.7117236919272036  to  0.8896546149090045
Step exceeds save_time, changing step size from  0.8896546149090045  to  0.7086588341733542
Curr Time:  6.291341165826646  Save time:  7.0 step:  0.7086588341733542
start time:  6.291341165826646 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.29441
Current:  [-0.04667321 -1.22304651  0.08463253] 
curr_time:  

Step exceeds save_time, changing step size from  0.770891376947338  to  0.6136482438040209
Curr Time:  6.386351756195979  Save time:  7.0 step:  0.6136482438040209
start time:  6.386351756195979 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.38942
Current:  [-0.04667321 -1.22304651  0.08492314] 
curr_time:  6.3894166139498285
AFTER: curr_time:  6.3894166139498285  current:  [-0.04667321 -1.22304651  0.08492314]
No reactions fired in this step (n= 0 ) changing step size from  0.6136482438040209  to  0.7670603047550262
Step exceeds save_time, changing step size from  0.7670603047550262  to  0.6105833860501715
Curr Time:  6.3894166139498285  Save time:  7.0 step:  0.6105833860501715
start time:  6.3894166139498285 step taken:  0.030648577538496195 number of steps: 11
Vode terminated at t=6.39248
Current:  [-0.04667321 -1.22304651  0.08493248] 
curr_time:  6.392481471703678
AFTER: curr_time:  6.392481471703678  current:  [-0.04667321 -1.22304651  0.08493248]
N

KeyboardInterrupt: 

In [7]:
plt.plot(results['time'], results['A'], label='A')
plt.plot(results['time'], results['V'], label='V')
plt.legend(loc='best')


NameError: name 'results' is not defined

In [ ]:
times_1 = [8.1, 7.06, 7.13, 7.04, 6.94, 6.94, 7.07, 6.63, 6.94, 7.23 ]
average_1 = sum(times_1) / len(times_1)
print(average_1)

In [ ]:
times_2 = [7.63, 6.98, 7.09, 7.12, 6.79, 6.75, 6.85, 7.1, 6.97, 6.98]
average_2 = sum(times_2) / len(times_2)
print(average_2)

In [ ]:
(1 - (average_2 / average_1)) * 100